### Промежуточная аттестация 3
#### Создать графическую программу, с использованием Tkinter, которая с выводит список имен, фамилий и их возвраст из базы данных sqlite. При запуске программы, должен создаваться(если ее нет) файл базы данных people.db и таблица соответственно. База данных должна заполняться в первый раз из словаря.Реализовать все проверки на исключения и максимально документировать весь код.

In [4]:
import sqlite3
import pandas as pd
from tkinter import * 
from tkinter import ttk
import datetime

first_name = None
last_name = None 
age = None
tree = None

#Функция подключения к БД
def connect_bd(db_filename = 'people.db'):
    try:
        conn = sqlite3.connect(db_filename) 
    except Exception as e:
        print(f'Ошибка соединения: {e}')
    return conn  

#Фунцкция создания таблицы и заполнения данными
def create_tbl():
    try:
        conn = connect_bd()
        cur = conn.cursor()
        cur.execute("""CREATE TABLE IF NOT EXISTS people( 
                first_name text,
                last_name text,
                age int,
                timestamp text);""") # Создание таблицы если ее нет
        now = datetime.datetime.now()
        if pd.read_sql('SELECT * FROM people;', conn).empty: #Проверка на пустоту таблицы
            people = [('Иван', 'Иванов', 18, now),
                     ('Анатолий', 'Смирнов', 56, now),
                     ('Андрей', 'Андреев', 37, now),
                     ('Алексей', 'Алексеев', 23, now),
                     ('Павел', 'Павлов', 28, now),
                     ('Антон', 'Антонов', 34, now),
                     ('Анна', 'Анина', 42, now),
                     ('Петр', 'Петров', 49, now),
                     ('Александра', 'Александрова', 51, now),
                     ('Демитрий', 'Дмитров', 25, now)] 
            cur.executemany("""INSERT INTO people(first_name, last_name, age, timestamp) 
                            VALUES(?, ?, ?, ?);""", people) # Добавление данных при первом запуске программы
        conn.commit()       
        conn.close()        
    except Exception as e:
        print(f'Ошибка: {e}')
        
#Функция добавления данных в таблицу        
def add_data():
    try:
        conn = connect_bd()
        cur = conn.cursor()
        people = (first_name.get(), last_name.get(), age.get(), datetime.datetime.now()) #Заполненные данные их ячеек
        cur.execute("""INSERT INTO people(first_name, last_name, age, timestamp)
                            VALUES(?, ?, ?, ?);""", people) # Вставка данных в таблицу 
        conn.commit()       
        conn.close()
        #Очистка ячеек от данных 
        first_name.delete(0, 'end')
        last_name.delete(0, 'end')
        age.delete(0, 'end')
        update_select()
    except Exception as e:
        print(f'Ошибка: {e}')
        
#Функция вывода содержимого таблицы people
def df_tbl():
    try:
        conn = connect_bd()
        create_tbl()
        #df = pd.read_sql('SELECT * FROM people;', conn)
        cur = conn.cursor()
        cur.execute('SELECT first_name, last_name, age FROM people ORDER BY timestamp DESC')
        df = cur.fetchall()   
    except Exception as e:
        print(f'Ошибка: {e}')
    return df

#Функция для обновления данных после внесея новых записей (в табличной части интерфейса)
def update_select():
    try:
        for item in tree.get_children():
            tree.delete(item)
        people = df_tbl()
        for person in people:
            tree.insert("", END, values=person)
    except Exception as e:
        print(f'Ошибка: {e}')

#Функция запуска интерфейса
def gui():
    global first_name, last_name, age, tree
    try:
        window = Tk()
        window.geometry('400x500')
        window.title('БД')
        
        #Возможность добавления данных в пользователем в БД
        text_first_name = Label(bg = "lavender", fg = "orchid4", text='Имя: ')
        first_name = Entry()
        text_last_name = Label(bg = "lavender", fg = "orchid4", text='Фамилия: ')
        last_name = Entry()
        text_age = Label(bg = "lavender", fg = "orchid4", text='Возраст: ')
        age = Entry()
        button = Button(window, bg = "thistle" , fg = "plum4", text = 'Сохранить в БД', font = "Arial 16", command = add_data)
         
        text_first_name.pack()
        first_name.pack()
        text_last_name.pack()
        last_name.pack()
        text_age.pack()
        age.pack()
        button.pack()
           
        #Вывод содержимого таблицы people
        columns = ("first_name", "last_name", "age")
        tree = ttk.Treeview(columns=columns, show="headings")
        tree.pack(fill=BOTH, expand=1)
        
        tree.heading("first_name", text="Имя", anchor=W)
        tree.heading("last_name", text="Фамилия", anchor=W)
        tree.heading("age", text="Возраст", anchor=W)
        
        tree.column("#1", stretch=NO, width=150)
        tree.column("#2", stretch=NO, width=150)
        tree.column("#3", stretch=NO, width=100)
        
        people = df_tbl()
        for person in people:
            tree.insert("", END, values=person)
        
        window.configure(bg = "lavender")
        window.mainloop()
    except Exception as e:
        print(f'Ошибка: {e}')
        
gui()